In [1]:
import sys
import random
import numpy as np
import os
from PIL import Image
from src.env.env import RILAB_OMY_ENV
import gc, argparse
import json
import copy
import time
from src.controllers import load_controller
from lerobot.datasets.lerobot_dataset import LeRobotDataset
import glfw

/opt/miniconda3/envs/mujoco2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment configuration
config_file_path = './configs/train_key_ur5.json'
with open(config_file_path) as f:
    env_conf = json.load(f)
language_instruction = env_conf['language_instruction']
omy_env = RILAB_OMY_ENV(cfg=env_conf, seed=None, 
                        action_type=env_conf['control_mode'], 
                        obs_type='eef_pose',
                        vis_mode = 'keyboard')


omy_env.reset(leader_pose = False)
# Load keyboard controller
controller = load_controller('keyboard',env_conf)
controller.reset(omy_env)


-----------------------------------------------------------------------------
name:[tabletop_env] dt:[0.002] HZ:[500]
 n_qpos:[38] n_qvel:[35] n_qacc:[35] n_ctrl:[8]
 integrator:[IMPLICITFAST]

n_body:[36]
 [0/36] [world] mass:[0.00]kg
 [1/36] [front_object_table] mass:[1.00]kg
 [2/36] [camera] mass:[0.00]kg
 [3/36] [camera2] mass:[0.00]kg
 [4/36] [camera3] mass:[0.00]kg
 [5/36] [base] mass:[4.00]kg
 [6/36] [shoulder_link] mass:[3.70]kg
 [7/36] [upper_arm_link] mass:[8.39]kg
 [8/36] [forearm_link] mass:[2.27]kg
 [9/36] [wrist_1_link] mass:[1.22]kg
 [10/36] [wrist_2_link] mass:[1.22]kg
 [11/36] [wrist_3_link] mass:[0.19]kg
 [12/36] [camera_center] mass:[0.00]kg
 [13/36] [attachment] mass:[0.00]kg
 [14/36] [base_mount] mass:[0.15]kg
 [15/36] [gripper_base] mass:[0.78]kg
 [16/36] [tcp_link] mass:[0.00]kg
 [17/36] [right_driver] mass:[0.01]kg
 [18/36] [right_coupler] mass:[0.01]kg
 [19/36] [right_spring_link] mass:[0.02]kg
 [20/36] [right_follower] mass:[0.01]kg
 [21/36] [right_pad] mass:

You can teleop your robot with keyboard
```
---------     -----------------------
   w       ->        backward
s  a  d        left   forward   right
---------      -----------------------
In x, y plane

---------
R: Moving Up
F: Moving Down
---------
In z axis

---------
Q: Tilt left
E: Tilt right
UP: Look Upward
Down: Look Donward
Right: Turn right
Left: Turn left
---------
For rotation

---------
SPACEBAR: Toggle Gripper
--------

---------
z: reset
--------
```

In [3]:
from src.dataset.utils import make_teleoperation_dataset

make_new = True
ROOT = './dataset/teleoperation_dataset'
if os.path.exists(ROOT):
    import shutil
    print("REMOVE")
    shutil.rmtree(ROOT)
    make_new = True
if make_new:
    print("CREATE")
    dataset = make_teleoperation_dataset(ROOT, state_dim=7)

REMOVE
CREATE


In [4]:
NUM_trials_PER_TASK = 5
episode_id = 0

In [5]:
# while omy_env.env.is_viewer_alive():
#     omy_env.step_env()
#     if omy_env.env.loop_every(HZ=20):
#         temp = omy_env.get_object_pose(pad=5)
#         delta_eef = controller.get_action()
#         omy_env.step(delta_eef)
#         omy_env.grab_image()
#         omy_env.render(task=language_instruction)
#         success = omy_env.check_success()
#         if omy_env.env.is_key_pressed_once(glfw.KEY_Z):
#             omy_env.reset(leader_pose = False)
#         if success or omy_env.env.is_key_pressed_once(glfw.KEY_ESCAPE):
#             break
#     omy_env.env.sync_sim_wall_time()
# omy_env.env.close_viewer()
# print("Episode ended.")
# print(f"Success: {success}")

In [ ]:
while omy_env.env.is_viewer_alive() and episode_id < NUM_trials_PER_TASK:
    omy_env.step_env()
    if omy_env.env.loop_every(HZ=20):
        key_list = omy_env.env.get_key_pressed_list()
        done = omy_env.check_success()
        if done or 90 in key_list:  # 'z' key to reset
            print("END EPISODE")
            if done:
                dataset.save_episode()
                episode_id += 1
            else: 
                dataset.clear_episode_buffer()
            omy_env.reset(leader_pose = False)
            action = controller.get_action()
            eef_pose = omy_env.step(action)
        action = controller.get_action()
        eef_pose = omy_env.step(action) 
        
        agent_image,wrist_image = omy_env.grab_image(return_side=False)
        # # resize to 256x256
        agent_image = Image.fromarray(agent_image)
        wrist_image = Image.fromarray(wrist_image)
        agent_image = agent_image.resize((256, 256))
        wrist_image = wrist_image.resize((256, 256))
        agent_image = np.array(agent_image)
        wrist_image = np.array(wrist_image)
        obj_states, recp_q_poses = omy_env.get_object_pose(pad=10)
        obj_poses = np.array(obj_states['poses'])
        dataset.add_frame( {
                "observation.image": agent_image,
                "observation.wrist_image": wrist_image,
                "observation.state": eef_pose,
                "action": action,
                "observation.eef_pose": eef_pose,
                'env.obj_pose': np.array(obj_states['poses'],dtype=np.float32),
                "env.obj_names": ','.join(obj_states['names']),
                "env.obj_q_names": ','.join(recp_q_poses['names']),
                "env.obj_q_states": np.array(recp_q_poses['poses'],dtype=np.float32),
                "env.config_file_name": config_file_path,
                "task": language_instruction
            }, 
        )
        last_obj_poses = obj_poses
        # based on the episode_id number, get the guide line
        omy_env.render(language_instruction, guideline= f' [Num Episode: {episode_id}/{NUM_trials_PER_TASK}]')
    omy_env.env.sync_sim_wall_time()
omy_env.env.close_viewer()
dataset.finalize()

END EPISODE


Map: 100%|██████████| 1870/1870 [00:01<00:00, 1492.22 examples/s]


['top', 'open']
DONE INITIALIZATION
END EPISODE


Map: 100%|██████████| 935/935 [00:00<00:00, 1557.46 examples/s]


['top', 'open']
DONE INITIALIZATION
END EPISODE
['top', 'open']
DONE INITIALIZATION
END EPISODE


Map: 100%|██████████| 720/720 [00:00<00:00, 1560.01 examples/s]


['top', 'open']
DONE INITIALIZATION
